In [1]:
from hydra import initialize
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from lightning.pytorch.loggers import WandbLogger
import wandb
import sys
from dataio.MimicCXRDataset import CXRDataset
from torchvision import transforms
sys.path.append("..")
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from dataio.MimicCXRDataset import train_test_split_CXR
from models.ResnetMimic import ResnetMimic
with initialize(version_base=None, config_path="../configs"):
       cfg_training = compose(config_name="basic_resnet_experiment")
       cfg_dataset = compose(config_name="dataset_config")
print(cfg_training)
print(cfg_dataset)

{}
{'root_dir': '/Users/ago/PycharmProjects/mml/data/mimic-cxr-preprocessed', 'class_names': ['No Finding'], 'trans_resize': 224, 'view_position': 'PA', 'splitting': {'method': 'random', 'train_val_split': 0.6, 'test_val_split': 0.5}, 'seed': 42, 'uncertain_values': 'exclude'}


In [2]:
torch.__version__

'2.1.2'

In [3]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [4]:
# create datasets
train_dict, test_dict, val_dict = train_test_split_CXR(cfg_dataset)

transformList = [transforms.RandomAffine(degrees=(0, 5), translate=(0.05, 0.05), shear=(5)),
                 transforms.RandomHorizontalFlip(), transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transform = transforms.Compose(transformList)

train_dataset = CXRDataset(label_name=cfg_dataset.class_names[0],labels_df=train_dict['labels'],images=train_dict['images'],transform=transform)

test_dataset = CXRDataset(label_name=cfg_dataset.class_names[0],labels_df=test_dict['labels'],images=test_dict['images'],transform=transform)

# create dataloaders
train_dl = DataLoader(train_dataset, batch_size=64, num_workers=9,
                         shuffle=False,
                         pin_memory=False)

test_dl = DataLoader(test_dataset, batch_size=64, num_workers=9,
                         shuffle=False,
                         pin_memory=False)

Root dir:  /Users/ago/PycharmProjects/mml/data/mimic-cxr-preprocessed
Image matrix shape:  (43082, 224, 224)
Number of metadata rows:  43082
Dropping readings with uncertain values in selected class names
Number of filtered metadata rows:  43082
Number of patients in the training set:  25849
Number of patients in the val set:  8617
Number of patients in the test set:  8616
Number of images in train set:  25849
Number of images in val set:  8617
Number of images in test set:  8616


In [ ]:
model = ResnetMimic()
wandb.finish()
if __name__ == '__main__':
    #init wandb_logger
    wandb.init()
    device='cpu'

    project_name = 'Resnet18_mimic'

    args = {
        'batch_size': 64,
        'learning_rate': 0.05,
        'project_name': project_name }

    wandb_logger = WandbLogger(project=project_name, log_model="all")
    wandb_logger.watch(model, log="all")

    trainer = pl.Trainer(max_epochs=3, logger=wandb_logger, accelerator='cpu')
    trainer.fit(model,train_dataloaders=train_dl,val_dataloaders=test_dl)
    wandb.finish()

/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
wandb: Currently logged in as: aa335 (aa95-team). Use `wandb login --relogin` to force relogin


/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name  | Type              | Params
--------------------------------------------
0 | model | ResNet            | 11.2 M
1 | loss  | BCEWithLogitsLoss | 0     
--------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.707    Total estima

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


In [35]:
# print matrix shape of the training data
for batch in train_dl:
    print(batch['image'].shape)
    print(batch['label'].shape)
    break


KeyboardInterrupt: 